<a href="https://colab.research.google.com/github/tktmyd/ipynbs/blob/main/fdm/FDM3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1次元問題の差分法シミュレーション：アルゴリズム

それでは，1次元の運動方程式と構成関係式

$$
\rho \frac{\partial v}{\partial t} = \frac{\partial \sigma}{\partial x}, \quad
\frac{\partial \sigma}{\partial t} = G \frac{\partial v}{\partial x}
$$

をスタガードグリッド差分法で表現してみましょう．ここでは簡単のため，密度$\rho$と剛性率$G$は定数であるとし，速度$v$と応力$\sigma$の変数を，それぞれ時間微分を時間ステップ$\Delta t$の，空間微分を空間グリッド間隔 $\Delta x$の，それぞれ中心差分で離散化していきます．

まずは先の例にならって空間微分を空間差分に置き換えてみましょう．
構成関係式と運動方程式の右辺の微分を差分に書き換えると，それぞれ

$$
\begin{align}
&\frac{\partial \sigma(x,t)}{\partial t} = G(x) \frac{v(x+\Delta x/2,t)-v(x-\Delta x / 2, t)}{\Delta x}
\\
& \frac{\partial v(x,t)}{\partial t} = \frac{1}{\rho(x) } \frac{\sigma(x+\Delta x/2,t)-\sigma(x-\Delta x / 2, t)}{\Delta x}
\end{align}
$$

と表されそうです．ただし，運動方程式の両辺を質量密度で割ることで，密度を右辺に移動しました．しかし，この組み合わせを直接コンピュータで解こうとすると困ったことがあります．構成関係式のほうに現れる応力 $\sigma$ の位置は $x$，変位速度 $v$の位置は $x\pm \Delta x/2$ なのに対して，運動方程式の位置は 応力が  $x\pm \Delta x/2$，変位速度が $x$ と，あべこべなのです．これでは，$x$と$x\pm \Delta x$における応力と変位速度が両方とも必要になってしまいます．

そこで，構成関係式の位置を全体に $-\Delta x/2$ だけ平行移動してみましょう．そうすると，


$$
\begin{align}
&\frac{\partial \sigma(x-\Delta x/2,t)}{\partial t} = G(x) \frac{v(x,t)-v(x-\Delta x, t)}{\Delta x}
\\
&\frac{\partial v(x,t)}{\partial t} = \frac{1}{\rho(x) } \frac{\sigma(x+\Delta x/2,t)-\sigma(x-\Delta x / 2, t)}{\Delta x}
\end{align}
$$

となり，位置の対応関係が改善しました．この式では，応力は $x\pm \Delta x/2$の位置だけに，変位速度は$x$と$x+\Delta x$の位置だけに現れます．そのうえで，$x$軸を間隔 $\Delta x$で等間隔に分割し，**グリッド位置** を

$$x_I = x_0 + I \Delta x$$

と定義します．ただし $x_0$ は座標系原点の基準位置，$x_I$ は $I$番目のグリッドにおける位置です．すると，
$$
v(x_I,t) = v_I(t), \quad \sigma(x_I-\Delta x/2, t) = \sigma_{I-1/2}
$$

と表されます．スタガードグリッド差分法では，変数によってグリッド配置が 1/2 分ずれるため，$I$は整数以外に半整数（整数$\pm 1/2$）の値を取りうるのです．これを構成関係式と運動方程式に代入すると，

$$
\begin{align}
	\frac{\partial \sigma_{I-1/2}(t)}{\partial t} = G_I \frac{v_{I}(t) - v_{I-1}(t) }{\Delta x}
	\\
	\frac{\partial v_I(t)}{\partial t} = \frac{1}{\rho_I } \frac{\sigma_{I+1/2}(t)- \sigma_{I-1/2}(t)}{\Delta x}
\end{align}
$$

となります．それぞれの式の左辺のグリッド位置（応力であれば $I-1/2$, 速度であれば $I$ を中心にするように，右辺の空間差分が構成されています．この方程式は任意の$I$について成立しますから，応力と速度それぞれについて等間隔（$\Delta x$）の離散的な位置における方程式が得られたことになります．

続けて，時間微分についても同じように離散化してみます．構成関係式の左辺には応力の時間微分がありますが，応力が時間に対して連続であるという仮定のもと，これを適当な **時間ステップ** 間隔 $\Delta t$ を用いて

$$
\begin{align}
	\frac{\partial \sigma_{I-1/2}(t)}{\partial t} \simeq \frac{\sigma_{I-1/2}(t+\Delta t/2)-\sigma_{I-1/2}(t-\Delta t/2)}{\Delta t}
\end{align}
$$

と近似します．

求めたいのは時刻 $t$ における微分値なので，その時刻を中心とした $t \pm \Delta t/2$ の時刻における $v_i$ を用いて微分値を近似しました．
もとの構成関係式に代入して整理すると，

$$
\begin{align}
	\sigma_{I-1/2}(t+\Delta t/2) = \sigma_{I-1/2}(t-\Delta t/2)
	+
	G_I \frac{v_{I}(t) - v_{I-1} (t) }{\Delta x} \Delta t
\end{align}
$$

と書けます．
この式は，時刻 $t-\Delta t/2$ における同じ位置の応力と時刻 $t$ における $I$, $I+1$ 番目の変位速度から $t+\Delta t/2$ における応力を求める，という，過去から未来を予測するような式になっています．

続けて，変位速度について検討します．
構成関係式の時間についての離散化結果からは，変位速度 $v$ は時刻 $t$, $t+\Delta t$, $t+2\Delta t$, $\cdots$ における値がわかれば良さそうです．
そこで，運動方程式については構成関係式から $\Delta t/2$ だけ時間をずらして，

$$
\begin{align}
	\frac{\partial v_I(t+\Delta t/2)}{\partial t}
	\simeq
	\frac{v_I(t+\Delta t) - v_I(t)}{\Delta t}
\end{align}
$$

とします．
すると，運動方程式は，

$$
\begin{align}
	v_I(t+\Delta t) = v_I(t)
	+
	\frac{1}{\rho_I}
	\frac{\sigma_{I+1/2}(t+\Delta t/2) - \sigma_{I-1/2}(t+\Delta t/2) }{\Delta x} \Delta t
\end{align}
$$

となります．

以上で，構成関係式と運動方程式ついて時間と空間の双方を離散化した式が得られました．
まとめると，

$$
\begin{align}
	 &	\sigma_{I-1/2}(t+\Delta t/2) = \sigma_{I-1/2}(t-\Delta t/2)
	 +
	 G_I\frac{v_{I}(t) - v_{I-1} (t) }{\Delta x} \Delta t
	\\
	 & v_I(t+\Delta t) = v_I(t)
	 +
	 \frac{1}{\rho_I}
	 \frac{\sigma_{I+1/2}(t+\Delta t/2) - \sigma_{I-1/2}(t+\Delta t/2) }{\Delta x} \Delta t
\end{align}
$$

の2式です．

応力と変位速度の離散化された時刻が互いに $\Delta t/2$ だけずれていて，かつ両辺とも右辺の値が左辺よりも過去の情報だけで記述されています．しかも，構成関係式によって，あらゆる $I$ について応力 $\sigma_{I-1/2}(t+\Delta t/2)$ を求めておけば，それが運動方程式の右辺に使えます．すなわち，これらの式を交互に使うことで，変位速度と応力の時間発展を追跡できるのです．

## グリッド変数の置き換え

これでスタガードグリッド差分法の核となるアルゴリズムの導出はほとんど終わりなのですが，この結果をそのままコンピュータプログラムに落とし込もうとすると，$I \pm 1/2 $の半整数のグリッド位置の扱いが厄介です．多くのプログラミング言語では，たとえば `v[1], v[2], .... ` のように一つの変数名 `v` に複数の値を割り当てる，配列あるいはリストといった概念がありますが，ほとんどの場合その引数は整数のみです．

そこで，このような半整数のグリッド位置を扱うために，グリッド位置を整数のみで表現する置き換えを考えます．

![fdm_cell.png](./fdm_cell.png)

そこで，グリッド番号$I$に対して，

$$
(I-1) \Delta x < x  \le I \Delta x
$$

の範囲を $i$ 番目の **セル** と定義します（上図）．大文字の $I$ がグリッド，小文字の $i$ はセルとして区別します．そうすると，グリッド番号 $I-1/2$ は $i$番目のセルに，$I+1/2$ は $i+1$ 番目のセルに属するわけですから，先の差分法アルゴリズムは，セル番号を用いて

$$
\begin{align}
	 &	\sigma_i(t+\Delta t/2) = \sigma_i(t-\Delta t/2)
	 +
	 G_i\frac{v_i(t) - v_{i-1} (t) }{\Delta x} \Delta t
	\\
	 & v_i(t+\Delta t) = v_i(t)
	 +
	 \frac{1}{\rho_i}
	 \frac{\sigma_{i+1}(t+\Delta t/2) - \sigma_{i}(t+\Delta t/2) }{\Delta x} \Delta t
\end{align}
$$

と，整数のセル番号のみを用いて書き下すことができました．

さらにもうひと工夫します．上式の差分法のアルゴリズムでは，右辺が過去，左辺が未来の値を表しています．たとえば応力$\sigma$に着目してみると，$t-\Delta t/2$における $\sigma$ の値がわかれば，$t+\Delta t/2$における値に**アップデート**できます．そして，一度アップデートしたら，$t-\Delta t/2$ の過去の値が参照されることはもうありません．

ということは，過去の値を保存しておく必要がないのです．そこで，過去の値を保存するための変数を導入せず，上書きしてしまえるのです．そこで，すべての変数から時間依存性を廃し，右辺をもって左辺を上書きする，ということを記号 $\leftarrow$ で表すことにします．


$$
\begin{align}
	 &	\sigma_i \leftarrow \sigma_i
	 +
	 G_i \frac{v_i - v_{i-1} }{\Delta x} \Delta t
	\\
	 & v_i  \leftarrow  v_i
	 +
	 \frac{1}{\rho_i }
	 \frac{\sigma_{i+1} - \sigma_{i} }{\Delta x} \Delta t
\end{align}
$$

これでアルゴリズムの完成です．これなら，そのままプログラムに落とし込むことができそうですね．プログラム言語の多くは，$\leftarrow $の役割は記号 `=` が担っています．たとえば，Pythonで

```python
i = 3
i = i + 1
```
と書くことができます．
2行目の式は，`=` を等号記号だと思うとおかしな式で，左辺と右辺が等しくないように見えてしまいますが，実際には，右辺の `i+1` という演算の結果をあらたに変数 `i` に割り当てる，という意味を持ちます．すなわち，変数 `i` のアップデートが容易にできるのです．もとの変数の値になにかを加えて，それを元の変数として再定義する，という操作は，プログラム言語でよく行われる操作で，Pythonでは上記のコードを

```python
i = 3
i += 1
```

というふうに `+=` 演算子を使って書くこともできます．

ところで，これまでの導出では，質量密度$\rho$と剛性率$G$は位置の関数として扱ってきました．その結果，上記のアルゴリズムでも，密度と剛性率はそれぞれセル番号$i$の付与された値になっています．ということは，密度と剛性率が空間的に変化するような**不均質**な場合においても，このアルゴリズムは有効なのです．このように，特別な工夫なく不均質構造を導入できることが，スタガードグリッド差分法の大きな強みの一つです．

> とはいえ，2次元以上で **厳密** に不均質媒質を扱うためには，グリッドあるいはセルの間の適切な平滑化が必要であることがわかっています．